In [87]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

import json

from pathlib import Path

In [88]:
DATA_PATH = '../datasets/kw_video/{video_id}/'
PRED_PATH = '../YOLOX_outputs/yolox_x_custom_kw/track_results/'
OUT_PATH = '../YOLOX_outputs/yolox_x_custom_kw/compare_230623/'
OUT_PATH_TRACK = '../YOLOX_outputs/yolox_x_custom_kw/track_230623/'
OUT_PATH_VIDEO = '../YOLOX_outputs/yolox_x_custom_kw/track_video_230623/'
ORIGIN_JSON_PATH = '../datasets/kw_video/origin_anno/'

video_list = [_.split('.txt')[0] for _ in os.listdir(PRED_PATH) if '.txt' in _]
json_list = [os.path.join(ORIGIN_JSON_PATH, _).replace('\\', '/') for _ in sorted(os.listdir(ORIGIN_JSON_PATH)) if 'sorted' in _]

Path(OUT_PATH_TRACK).mkdir(parents=True, exist_ok=True)
Path(OUT_PATH_VIDEO).mkdir(parents=True, exist_ok=True)

In [89]:
def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    
    return iou

In [90]:
def get_color(idx):
    idx = idx * 3
    color = ((37 * idx) % 255, (17 * idx) % 255, (29 * idx) % 255)

    return color

In [92]:
blue = (255,0,0)
red = (0,0,255)
green = (0,255,0)
font=cv2.FONT_HERSHEY_SIMPLEX

text_scale = 1
text_thickness = 2
line_thickness = 2

print(video_list)
for video_id in video_list:
    data_path = DATA_PATH.format(video_id = video_id)
    image_path = os.path.join(data_path, 'images').replace('\\', '/')
    pred_file = os.path.join(PRED_PATH, video_id+'.txt')
    
    if not os.path.exists(os.path.join(OUT_PATH, video_id)):
        os.mkdir(os.path.join(OUT_PATH, video_id))
        
    pred = np.loadtxt(pred_file, delimiter=",")
    image_list = [_ for _ in sorted(os.listdir(image_path)) if '.png' in _]
    
    pred_dict = {}
    head_dict = {}
    prev_frame = 0
    for pre in pred:
        frame_id = int(pre[0])
        if prev_frame != frame_id:
            pred_dict[frame_id] = []
            head_dict[frame_id] = []
            prev_frame = frame_id

        _id, _x, _y, _w, _h, _score  = pre[1:7]

        pred_dict[frame_id].append([int(_id), int(_x), int(_y), int(_w), int(_h), _score])
#         head_dict[frame_id].append([int(_id), int(_x), int(_y), int(_w), int(_h*0.25)])
        
    for image_idx, image_filename in enumerate(image_list):
        image_full_path = os.path.join(image_path, image_filename)
        
        image = cv2.imread(image_full_path)
        
        pre = pred_dict[image_idx+1]
#         head = head_dict[image_idx+1]
        
        cv2.putText(image, 'frame: %d num: %d' % (image_idx+1, len(pre)),
                    (0, int(30 * text_scale)), cv2.FONT_HERSHEY_PLAIN, 2, red, thickness=2)
        
        for p in pre:
#         for (p, h) in zip(pre, head):
            _id = p[0]
            color = get_color(abs(_id))
            _bbox = p[1:5]
            _bbox[2] = _bbox[0] + _bbox[2]
            _bbox[3] = _bbox[1] + _bbox[3]
            _score = round(p[5] * 100, 0)
            
#             _hbbox = h[1:5]
#             _hbbox[2] = _hbbox[0] + _hbbox[2]
#             _hbbox[3] = _hbbox[1] + _hbbox[3]
#             _hscore = round(random.uniform(0.9, 1)*100, 0)
            
            bounding_image = cv2.rectangle(image, (_bbox[0], _bbox[1]), (_bbox[2], _bbox[3]), color=color, thickness=line_thickness)
#             bounding_image = cv2.rectangle(image, (_hbbox[0], _hbbox[1]), (_hbbox[2], _hbbox[3]), color=color, thickness=line_thickness)
            bounding_image = cv2.putText(bounding_image, 'Pred_ID: %d' % (_id), (_bbox[0], _bbox[1]), font, text_scale, color=color, thickness=text_thickness)
            bounding_image = cv2.putText(bounding_image, 'Score: %2d' % (_score), (_bbox[0], _bbox[3]+30), font, text_scale, color=color, thickness=text_thickness)
#             bounding_image = cv2.putText(bounding_image, 'Head_Direction(Score): Left(%2d)' % (_hscore), (_hbbox[0]+60, _hbbox[3]+30), font, text_scale, color=color, thickness=text_thickness)
            
        
        cv2.imwrite(os.path.join(OUT_PATH, video_id, image_filename), bounding_image)

['video_01', 'video_04']


In [93]:
_y, _h, (_y-_h)*0.25

(402.8, 40.4, 90.60000000000001)

In [95]:
blue = (255,0,0)
red = (0,0,255)
green = (0,255,0)
font=cv2.FONT_HERSHEY_SIMPLEX

text_scale = 1
text_thickness = 2
line_thickness = 2

target_id_dict = {
    0: [1, 11],
    1: [1],
}

target_tcp = {
    0: {},
    1: {}
}

fps = 1/30

for idx, video_id in enumerate(video_list[1:]):
    data_path = DATA_PATH.format(video_id = video_id)
    image_path = os.path.join(data_path, 'images').replace('\\', '/')
    pred_file = os.path.join(PRED_PATH, video_id+'.txt')
    
    if not os.path.exists(os.path.join(OUT_PATH_TRACK, video_id)):
        os.mkdir(os.path.join(OUT_PATH_TRACK, video_id))
        
    pred = np.loadtxt(pred_file, delimiter=",")
    image_list = [_ for _ in sorted(os.listdir(image_path)) if '.png' in _]
    
    target_list = target_id_dict[idx]
    json_filepath = json_list[idx]
    
    with open(json_filepath, 'r') as json_file:
        json_data = json.load(json_file) 
        
    crosswalk_dict = {}

    for anno in json_data['annotations']:
        _category_id = anno['category_id']

        if _category_id != 1:
            continue

        _image_id = anno['image_id'] + 1
        _bbox = anno['bbox']

        crosswalk_dict[_image_id] = _bbox
        
    pred_dict = {}
    prev_frame = 0
    head_dict = {}
    for pre in pred:
        frame_id = int(pre[0])
        if prev_frame != frame_id:
            pred_dict[frame_id] = []
            prev_frame = frame_id
#             head_dict[frame_id] = []

        _id, _x, _y, _w, _h, _score  = pre[1:7]

        pred_dict[frame_id].append([int(_id), int(_x), int(_y), int(_w), int(_h), _score])
#         head_dict[frame_id].append([int(_id), int(_x), int(_y), int(_w), int(_h*0.25)])
        
        
    target_tcp[idx][target_list[0]] = []
    for image_idx, image_filename in enumerate(image_list):
        image_full_path = os.path.join(image_path, image_filename)
        
        image = cv2.imread(image_full_path)
        
        pre = pred_dict[image_idx+1]
#         head = head_dict[image_idx+1]
        
        cv2.putText(image, 'frame: %d num: %d' % (image_idx+1, len(pre)),
                    (0, int(30 * text_scale)), cv2.FONT_HERSHEY_PLAIN, 2, red, thickness=2)
        
        cross_walk_bbox = crosswalk_dict[image_idx+1]
        cross_walk_bbox[2] = cross_walk_bbox[0] + cross_walk_bbox[2]
        cross_walk_bbox[3] = cross_walk_bbox[1] + cross_walk_bbox[3]
        cv2.rectangle(image, (cross_walk_bbox[0], cross_walk_bbox[1]), (cross_walk_bbox[2], cross_walk_bbox[3]), color=green, thickness=line_thickness)
        
        for (h, p) in zip(head, pre):
            _id = p[0]
            
            if _id != target_list[0]: # if _id not in target_list:
                continue
            
            
            color = get_color(abs(_id))
            _bbox = p[1:5]
            _bbox[2] = _bbox[0] + _bbox[2]
            _bbox[3] = _bbox[1] + _bbox[3]
            _score = round(p[5] * 100, 0)
            
#             _hbbox = h[1:5]
#             _hbbox[2] = _hbbox[0] + _hbbox[2]
#             _hbbox[3] = _hbbox[1] + _hbbox[3]
#             _hscore = round(random.uniform(0.9, 1)*100, 0)
            
            bounding_image = cv2.rectangle(image, (_bbox[0], _bbox[1]), (_bbox[2], _bbox[3]), color=color, thickness=line_thickness)
#             bounding_image = cv2.rectangle(image, (_hbbox[0], _hbbox[1]), (_hbbox[2], _hbbox[3]), color=color, thickness=line_thickness+1)
            bounding_image = cv2.putText(bounding_image, 'Pred_ID: %d' % (_id), (_bbox[0], _bbox[1]), font, text_scale, color=color, thickness=text_thickness)
            bounding_image = cv2.putText(bounding_image, 'Score: %2d' % (_score), (_bbox[0], _bbox[3]+30), font, text_scale, color=color, thickness=text_thickness)
#             bounding_image = cv2.putText(bounding_image, 'Head(Score): Left(%2d)' % (_hscore), (_hbbox[2], _hbbox[3]), font, text_scale, color=color, thickness=text_thickness)
            
            if image_idx >= 1:
                x_lu = _bbox[0]
                y_lu = _bbox[1]
                x_rd = _bbox[2]
                y_rd = _bbox[3]
                
                x_lu_pr = prev_box[0]
                y_lu_pr = prev_box[1]
                x_rd_pr = prev_box[2]
                y_rd_pr = prev_box[3]                
                
                prev_box = _bbox
                
                xc_ru = cross_walk_bbox[2]
                yc_ru = cross_walk_bbox[1]
                
                iou = IoU(cross_walk_bbox, _bbox)
                
                if xc_ru >= x_rd and yc_ru <=y_rd or iou !=0:
                    bounding_image = cv2.putText(bounding_image, 'Crossing', (_bbox[0], _bbox[3]+60), font, text_scale, color, text_thickness)
                    cv2.imwrite(os.path.join(OUT_PATH_TRACK, video_id, image_filename), bounding_image)
                    continue
                
                if (x_lu - x_lu_pr) < -1:
                    v = 'LEFT'
                    d_cp = math.sqrt((xc_ru - x_rd)**2+(yc_ru-y_rd)**2)+1e-10
                elif (x_lu - x_lu_pr) > 1:
                    v = 'RIGHT'
                else:
                    v = 'STOP'
                
                d = math.sqrt((x_rd-x_rd_pr)**2 + (y_rd-y_rd_pr)**2)+1e-10
                
                if image_idx >=2:
                    s = abs((d-d_pr)/(image_idx+1-image_idx))
                    a = (s - s_pr)/(image_idx+1-image_idx)
#                     a = max(1, a)
                    if v == 'STOP':
                        a = 0
                    
                else:
                    s = d
                    a = s
                
#                 t_cp = (d_cp/(d+a)*fps) if v != 'STOP' else 0.
                t_cp = (d_cp/d*fps) if v != 'STOP' else 0.
                prev_t_cp = t_cp
                d_pr = d
                s_pr = s
                
            else:
                prev_box = _bbox
                prev_t_cp = 0
                x_lu = _bbox[0]
                y_lu = _bbox[1]
                x_rd = _bbox[2]
                y_rd = _bbox[3]
                
                v = 'STOP'
                s = 0.
                d = 0.
                d_pr = 0.
                s_pr = 0.
                a = 0.
                t_cp = 0.
            
            target_tcp[idx][target_list[0]].append(t_cp)
                
            bounding_image = cv2.putText(bounding_image, 'v: %s    s: %.2f    a: %.2f' % (v, s, a), (_bbox[0], _bbox[3]+60), font, text_scale, color, text_thickness)
            bounding_image = cv2.putText(bounding_image, 'Tcp: %.2fsecond' % (t_cp), (_bbox[0], _bbox[3]+90), font, text_scale, color, text_thickness)
        
        cv2.imwrite(os.path.join(OUT_PATH_TRACK, video_id, image_filename), bounding_image)
        
    
#     if idx == 0:
#         target_tcp[idx][target_list[1]] = [] 
#         for image_idx, image_filename in enumerate(image_list):
#             image_full_path = os.path.join(OUT_PATH_TRACK, video_id, image_filename)

#             image = cv2.imread(image_full_path)

#             pre = pred_dict[image_idx+1]

#             cross_walk_bbox = crosswalk_dict[image_idx+1]
#             cross_walk_bbox[2] = cross_walk_bbox[0] + cross_walk_bbox[2]
#             cross_walk_bbox[3] = cross_walk_bbox[1] + cross_walk_bbox[3]

#             for (h, p) in zip(head, pre):
#                 _id = p[0]

#                 if _id != target_list[1]:
#                     continue

#                 color = get_color(abs(_id))
#                 _bbox = p[1:5]
#                 _bbox[2] = _bbox[0] + _bbox[2]
#                 _bbox[3] = _bbox[1] + _bbox[3]
#                 _score = round(p[5] * 100, 0)

                
#                 _hbbox = h[1:5]
#                 _hbbox[2] = _hbbox[0] + _hbbox[2]
#                 _hbbox[3] = _hbbox[1] + _hbbox[3]
#                 _hscore = round(random.uniform(0.9, 1)*100, 0)
            
#                 bounding_image = cv2.rectangle(image, (_bbox[0], _bbox[1]), (_bbox[2], _bbox[3]), color=color, thickness=line_thickness)
#                 bounding_image = cv2.rectangle(image, (_hbbox[0], _hbbox[1]), (_hbbox[2], _hbbox[3]), color=color, thickness=line_thickness+1)
#                 bounding_image = cv2.putText(bounding_image, 'Pred_ID: %d' % (_id), (_bbox[0], _bbox[1]), font, text_scale, color=color, thickness=text_thickness)
#                 bounding_image = cv2.putText(bounding_image, 'Score: %2d' % (_score), (_bbox[0], _bbox[3]+30), font, text_scale, color=color, thickness=text_thickness)
#                 bounding_image = cv2.putText(bounding_image, 'Head(Score): Left(%2d)' % (_hscore), (_hbbox[2], _hbbox[3]), font, text_scale, color=color, thickness=text_thickness)

#                 if image_idx >= 1:
#                     x_lu = _bbox[0]
#                     y_lu = _bbox[1]
#                     x_rd = _bbox[2]
#                     y_rd = _bbox[3]

#                     x_lu_pr = prev_box[0]
#                     y_lu_pr = prev_box[1]
#                     x_rd_pr = prev_box[2]
#                     y_rd_pr = prev_box[3]                

#                     prev_box = _bbox

#                     xc_ru = cross_walk_bbox[2]
#                     yc_ru = cross_walk_bbox[1]

#                     iou = IoU(cross_walk_bbox, _bbox)

#                     if xc_ru >= x_rd and yc_ru <=y_rd or iou !=0:
#                         bounding_image = cv2.putText(bounding_image, 'Crossing', (_bbox[0], _bbox[3]+60), font, text_scale, color, text_thickness)
#                         cv2.imwrite(os.path.join(OUT_PATH_TRACK, video_id, image_filename), bounding_image)
#                         continue

#                     if (x_lu - x_lu_pr) < -1:
#                         v = 'LEFT'
#                         d_cp = math.sqrt((xc_ru - x_rd)**2+(yc_ru-y_rd)**2)+1e-10
#                     elif (x_lu - x_lu_pr) > 1:
#                         v = 'RIGHT'
#                     else:
#                         v = 'STOP'

#                     d = math.sqrt((x_rd-x_rd_pr)**2 + (y_rd-y_rd_pr)**2)+1e-10

#                     if image_idx >=2:
#                         s = abs((d-d_pr)/(image_idx+1-image_idx))
#                         a = (s - s_pr)/(image_idx+1-image_idx)
#                         a = max(1.0, a)

#                     else:
#                         s = d
#                         a = s

#                     t_cp = d_cp/d*fps if v != 'STOP' else 0.
#                     d_pr = d
#                     s_pr = s

#                 else:
#                     prev_box = _bbox

#                     x_lu = _bbox[0]
#                     y_lu = _bbox[1]
#                     x_rd = _bbox[2]
#                     y_rd = _bbox[3]

#                     v = 'STOP'
#                     s = 0.
#                     d = 0.
#                     d_pr = 0.
#                     s_pr = 0.
#                     a = 0.
#                     t_cp = 0.
                
#                 target_tcp[idx][target_list[1]].append(t_cp)
                
#                 bounding_image = cv2.putText(bounding_image, 'v: %s    s: %.2f    a: %.2f' % (v, s, a), (_bbox[0], _bbox[3]+60), font, text_scale, color, text_thickness)
#                 bounding_image = cv2.putText(bounding_image, 'Tcp: %.2fsecond' % (t_cp), (_bbox[0], _bbox[3]+90), font, text_scale, color, text_thickness)

#                 cv2.imwrite(os.path.join(OUT_PATH_TRACK, video_id, image_filename), bounding_image)

In [54]:
import re
path = os.path.join(OUT_PATH_TRACK, video_list[1])
paths = [os.path.join(path , i ) for i in os.listdir(path) if re.search(".png$", i )]
# pathIn= os.path.join(OUT_PATH_TRACK, 'crosswalk')
fps = 30
pathOut = os.path.join(OUT_PATH_VIDEO, '%s_fps%02d.mp4' % (video_list[1], fps))

import cv2
frame_array = []
for idx , path in enumerate(paths): 
    img = cv2.imread(path)
    height, width, layers = img.shape
    size = (width,height)
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [51]:
pathOut

'../YOLOX_outputs/yolox_x_custom_kw/track_video_230622/video_04\\video_04_fps15.mp4'

In [55]:
true_tcp = {
    0: {},
    1: {}
}

In [70]:
len(target_tcp[0][1])

148

In [96]:
rest_time=[]

fps_1=1/30

for i in range(0,len(target_tcp[0][1]),1):
    if i < 125:
        i = 125-i
    else:
        i = 0
    
    i=i*fps_1
    rest_time.append(i)
    print(i)

4.166666666666667
4.133333333333333
4.1
4.066666666666666
4.033333333333333
4.0
3.966666666666667
3.933333333333333
3.9
3.8666666666666667
3.8333333333333335
3.8
3.7666666666666666
3.7333333333333334
3.6999999999999997
3.6666666666666665
3.6333333333333333
3.6
3.5666666666666664
3.533333333333333
3.5
3.466666666666667
3.433333333333333
3.4
3.3666666666666667
3.3333333333333335
3.3
3.2666666666666666
3.2333333333333334
3.2
3.1666666666666665
3.1333333333333333
3.1
3.0666666666666664
3.033333333333333
3.0
2.966666666666667
2.933333333333333
2.9
2.8666666666666667
2.8333333333333335
2.8
2.7666666666666666
2.7333333333333334
2.7
2.6666666666666665
2.6333333333333333
2.6
2.5666666666666664
2.533333333333333
2.5
2.466666666666667
2.433333333333333
2.4
2.3666666666666667
2.3333333333333335
2.3
2.2666666666666666
2.2333333333333334
2.2
2.1666666666666665
2.1333333333333333
2.1
2.0666666666666664
2.033333333333333
2.0
1.9666666666666666
1.9333333333333333
1.9
1.8666666666666667
1.83333333333333

In [97]:
len(rest_time)

148

In [58]:
rest_time2=[]

fps_1=1/30

for i in range(0,296):
    if i <= 50 or i > 197:
        i = 0
    if i >= 50 and i <= 197:
        i = 197 - i
        
    i=i*fps_1
    rest_time2.append(i)
    print(i)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.866666666666666
4.833333333333333
4.8
4.766666666666667
4.733333333333333
4.7
4.666666666666667
4.633333333333333
4.6
4.566666666666666
4.533333333333333
4.5
4.466666666666667
4.433333333333334
4.4
4.366666666666666
4.333333333333333
4.3
4.266666666666667
4.233333333333333
4.2
4.166666666666667
4.133333333333333
4.1
4.066666666666666
4.033333333333333
4.0
3.966666666666667
3.933333333333333
3.9
3.8666666666666667
3.8333333333333335
3.8
3.7666666666666666
3.7333333333333334
3.6999999999999997
3.6666666666666665
3.6333333333333333
3.6
3.5666666666666664
3.533333333333333
3.5
3.466666666666667
3.433333333333333
3.4
3.3666666666666667
3.3333333333333335
3.3
3.2666666666666666
3.2333333333333334
3.2
3.1666666666666665
3.1333333333333333
3.1
3.0666666666666664
3.033333333333333
3.0
2.96666

In [59]:
rest_time3=[]

fps_1=1/30

for i in range(0,230):
    if i < 128 and i != 0:
        i = 128-i
    else:
        i = 0
    
    i=i*fps_1
    rest_time3.append(i)
    print(i)

0.0
4.233333333333333
4.2
4.166666666666667
4.133333333333333
4.1
4.066666666666666
4.033333333333333
4.0
3.966666666666667
3.933333333333333
3.9
3.8666666666666667
3.8333333333333335
3.8
3.7666666666666666
3.7333333333333334
3.6999999999999997
3.6666666666666665
3.6333333333333333
3.6
3.5666666666666664
3.533333333333333
3.5
3.466666666666667
3.433333333333333
3.4
3.3666666666666667
3.3333333333333335
3.3
3.2666666666666666
3.2333333333333334
3.2
3.1666666666666665
3.1333333333333333
3.1
3.0666666666666664
3.033333333333333
3.0
2.966666666666667
2.933333333333333
2.9
2.8666666666666667
2.8333333333333335
2.8
2.7666666666666666
2.7333333333333334
2.7
2.6666666666666665
2.6333333333333333
2.6
2.5666666666666664
2.533333333333333
2.5
2.466666666666667
2.433333333333333
2.4
2.3666666666666667
2.3333333333333335
2.3
2.2666666666666666
2.2333333333333334
2.2
2.1666666666666665
2.1333333333333333
2.1
2.0666666666666664
2.033333333333333
2.0
1.9666666666666666
1.9333333333333333
1.9
1.8666666

In [98]:
true_tcp[0][1] = rest_time
# true_tcp[0][11] = rest_time2
# true_tcp[1][1] = rest_time3


In [99]:
import copy

In [100]:
new_tartget_tcp = copy.deepcopy(target_tcp)

In [101]:
new_tartget_tcp[0][1]

[0.0,
 10.405265760464875,
 16.429056237373914,
 5.602718760996867,
 5.570446057502899,
 4.560175435131226,
 7.178749504001393,
 7.14724500125016,
 0.0,
 22.60199106047918,
 0.0,
 0.0,
 0.0,
 11.284330334513216,
 7.489556514667643,
 5.417232247868225,
 3.6390318152114487,
 5.336396179223749,
 5.467301792237505,
 10.901274134977541,
 7.685195001954343,
 10.83517932936078,
 6.821355355496471,
 4.280877376237701,
 21.37106038854928,
 9.527329111136536,
 21.270428087388524,
 15.017360323282132,
 6.68457594412268,
 7.012836554486837,
 5.070083986754876,
 3.4507558663269164,
 4.1075810669483825,
 4.074255214747477,
 5.059493277093146,
 3.9361788923906538,
 3.9808039388258543,
 4.795272726526425,
 4.90952871679403,
 196381148676.67075,
 19.60478966224935,
 8.783140414947367,
 19.60637651182004,
 4.868271710090508,
 8.678453522883268,
 3.8477987934621094,
 2.7151043042601364,
 3.134298511656133,
 3.655612708286382,
 3.694686875848887,
 3.661365865284247,
 3.557777083697903,
 2.990210364572472,

In [102]:
true_tcp

{0: {1: [4.166666666666667,
   4.133333333333333,
   4.1,
   4.066666666666666,
   4.033333333333333,
   4.0,
   3.966666666666667,
   3.933333333333333,
   3.9,
   3.8666666666666667,
   3.8333333333333335,
   3.8,
   3.7666666666666666,
   3.7333333333333334,
   3.6999999999999997,
   3.6666666666666665,
   3.6333333333333333,
   3.6,
   3.5666666666666664,
   3.533333333333333,
   3.5,
   3.466666666666667,
   3.433333333333333,
   3.4,
   3.3666666666666667,
   3.3333333333333335,
   3.3,
   3.2666666666666666,
   3.2333333333333334,
   3.2,
   3.1666666666666665,
   3.1333333333333333,
   3.1,
   3.0666666666666664,
   3.033333333333333,
   3.0,
   2.966666666666667,
   2.933333333333333,
   2.9,
   2.8666666666666667,
   2.8333333333333335,
   2.8,
   2.7666666666666666,
   2.7333333333333334,
   2.7,
   2.6666666666666665,
   2.6333333333333333,
   2.6,
   2.5666666666666664,
   2.533333333333333,
   2.5,
   2.466666666666667,
   2.433333333333333,
   2.4,
   2.3666666666666667,

In [63]:
first_time = [0 for _ in range(50)]
second_time = [0 for _ in range(98)]
third_time = [0 for _ in range(102)]

In [103]:
new_target_tcp = copy.deepcopy(target_tcp)
new_target_tcp[0][1] = new_target_tcp[0][1]
# new_target_tcp[0][11] = first_time + new_target_tcp[0][11] +second_time
# new_target_tcp[1][1] = new_target_tcp[1][1] +third_time

In [104]:
import pandas as pd

for true, tar in zip(true_tcp.items(), new_target_tcp.items()):
    _true = true[1]
    _tar = tar[1]
    
    for _t, _c in zip(_true.items(), _tar.items()):
        df = pd.DataFrame(columns=['True', 'T_CP'])
        df['True'] = _t[1]
        df['T_CP'] = _c[1]
        
        df.to_excel('../video_230623_%d_id_%d.xlsx' % (true[0], _t[0]))
        
#     for _true, _tar in zip(true[1].items(), tar[1].items()):
#         df = pd.DataFrame(columns=['True', 'T_CP'])
#         df['True'] = _true[1]
#         df['T_CP'] = _tar[1]
        
#         df.to_csv('../video_%d_id_%d' % (true[0], _true[0]))
#     break

In [487]:
_t[0]

1

In [415]:
true[1]#[0]#, _t[0]

{1: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0

In [397]:
_tar

{1: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  4.233333333333333,
  4.2,
  4.166666666666667,
  4.133333333333333,
  4.1,
  4.066666666666666,
  4.033333333333333,
  4.0,
  3.966666666666667,
  3.933333333333333,
  3.9,
  3.8666666666666667,
  3.8333333333333335,
  3.8,
  3.7666666666666666,
  3.7333333333333334,
  3.69

In [372]:
len(_true[1])

296

In [379]:
len(true_tcp[1][1])

231

In [380]:
len(new_target_tcp[1][1])

231

In [322]:
tar

(1,
 {1: [0.0,
   8.326517212058867,
   13.142911058313699,
   4.475306247856968,
   4.443376505574483,
   3.6308064606961756,
   5.709738269751186,
   5.678693120210672,
   0.0,
   17.95936771783742,
   0.0,
   0.0,
   0.0,
   8.963041398047173,
   5.9420763713198435,
   4.291654768583826,
   2.8758965176987843,
   4.21094124192413,
   4.307261310749507,
   8.581245700388878,
   6.045751492456567,
   8.516715589781038,
   5.35555786059653,
   3.353889947111012,
   16.736188333790093,
   7.453932593347205,
   16.634235110835398,
   11.740196855367678,
   5.219535951963882,
   5.4686197954469495,
   3.946190352720532,
   2.6785038203842855,
   3.1809502423087266,
   3.14769757117927,
   3.9013708416004125,
   3.027424506894204,
   3.0541265344644537,
   3.671992566865651,
   3.7517588466940293,
   150070353871.5129,
   14.973791917871083,
   6.713601450818558,
   14.978836921162547,
   3.711478067261327,
   6.608412147197293,
   2.922137725801306,
   2.054009960049169,
   2.363116795139

In [239]:
target_tcp[0].keys()

dict_keys([1, 11])

In [243]:
len(target_tcp)

2

In [125]:
# blue = (255,0,0)
# red = (0,0,255)
# green = (0,255,0)
# font=cv2.FONT_HERSHEY_SIMPLEX





# cross_walk_dict = {
#     'crosswalk': [[35, 582, 928, 836]],
#     'fourway': [[1133, 625, 1920, 1045], [907, 561, 1545, 629], [110, 565, 756, 601], [63, 591, 854, 967]],
#     'night': None
# }

# for data_idx, sub_path in enumerate(data_list):
#     image_path = os.path.join(DATA_PATH, sub_path, 'images')
#     anno_file = os.path.join(DATA_PATH, sub_path, 'labels_with_ids', sub_path+'.txt')
#     pred_file = os.path.join(PRED_PATH, sub_path+'.txt')
    
    
#     if not os.path.exists(os.path.join(OUT_PATH, sub_path)):
#         os.mkdir(os.path.join(OUT_PATH, sub_path))
    
#     anno = np.loadtxt(anno_file, delimiter=",", dtype=int)
#     anno[anno<0]=0
    
#     pred = np.loadtxt(pred_file, delimiter=",")
    
#     cross_walk_box = cross_walk_dict[sub_path]
    
#     image_list = sorted(os.listdir(image_path))
    
#     t = np.array([_+1 for _ in range(len(image_list))])
#     y = pred[:,0]
    
#     pred_idx = np.isin(t,y)
#     pred_c = 0
#     for image_idx, image_filename in enumerate(image_list):
#         image_full_path = os.path.join(image_path, image_filename)
#         box = anno[image_idx, 2:]
#         box[2] = box[0]+box[2]
#         box[3] = box[1]+box[3]
        
#         image = cv2.imread(image_full_path)
#         bounding_image = cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), green, 5)
#         bounding_image = cv2.putText(bounding_image, 'GT', (box[0]-30, box[1]-30), font, 2, green, 3)
        
#         if pred_idx[image_idx]:
#             pred_box = [int(_) for _ in pred[pred_c, 2:6]]
#             pred_box[2] = pred_box[0]+pred_box[2]
#             pred_box[3] = pred_box[1]+pred_box[3]
#             score = pred[pred_c, 6]*100
#             bounding_image = cv2.rectangle(bounding_image, (pred_box[0], pred_box[1]), (pred_box[2], pred_box[3]), blue, 3)
#             bounding_image = cv2.putText(bounding_image, 'Pred', (pred_box[0], pred_box[3]+60), font, 1, blue, 1)
#             bounding_image = cv2.putText(bounding_image, 'Score: %2d' % (score), (pred_box[0], pred_box[3]+120), font, 1, blue, 13)
#             pred_c+=1
        
#         if cross_walk_box is not None:
#             for i in range(len(cross_walk_box)):
#                 c_box = cross_walk_box[i]
#                 bounding_image = cv2.rectangle(bounding_image, (c_box[0], c_box[1]), (c_box[2], c_box[3]), red, 5)
#                 if i == 1 or i == 2:
#                     bounding_image = cv2.putText(bounding_image, 'Crosswalk', (c_box[0]+30, c_box[1]-40), font, 2, red, 3)
#                 else:
#                     bounding_image = cv2.putText(bounding_image, 'Crosswalk', (c_box[0]+30, c_box[1]+80), font, 2, red, 3)
        
#         cv2.imwrite(os.path.join(OUT_PATH, sub_path, image_filename), bounding_image)


In [127]:
# blue = (255,0,0)
# red = (0,0,255)
# green = (0,255,0)
# black = (0,0,0)
# white = (255,255,255)
# fps = 1/30
# font=cv2.FONT_HERSHEY_SIMPLEX

# cross_walk_dict = {
#     'crosswalk': [[35, 570, 928, 836]],
#     'fourway': [[1133, 625, 1920, 1045], [907, 561, 1545, 629], [110, 565, 756, 601], [63, 591, 854, 967]],
#     'night': None
# }

# for data_idx, sub_path in enumerate(data_list):
#     if data_idx >=11:
#         continue
    
#     image_path = os.path.join(DATA_PATH, sub_path, 'images')
#     anno_file = os.path.join(DATA_PATH, sub_path, 'labels_with_ids', sub_path+'.txt')
#     pred_file = os.path.join(PRED_PATH, sub_path+'.txt')
    
    
#     if not os.path.exists(os.path.join(OUT_PATH_TRACK, sub_path)):
#         os.mkdir(os.path.join(OUT_PATH_TRACK, sub_path))
    
#     anno = np.loadtxt(anno_file, delimiter=",", dtype=int)
#     anno[anno<0]=0
    
#     pred = np.loadtxt(pred_file, delimiter=",")
#     pred[pred<0] = 0
    
#     cross_walk_box = cross_walk_dict[sub_path]
    
#     image_list = sorted(os.listdir(image_path))
    
#     t = np.array([_+1 for _ in range(len(image_list))])
#     y = pred[:,0]
    
#     pred_idx = np.isin(t,y)
#     pred_c = 0
#     for image_idx, image_filename in enumerate(image_list):
#         image_full_path = os.path.join(image_path, image_filename)
#         image = cv2.imread(image_full_path)
        
#         bounding_image = cv2.putText(image, 'Frame: %4d' % (image_idx + 1), (35, 100), font, 3, black, 4)
#         if cross_walk_box is not None:
#             for i in range(len(cross_walk_box)):
#                 c_box = cross_walk_box[i]
#                 bounding_image = cv2.rectangle(bounding_image, (c_box[0], c_box[1]), (c_box[2], c_box[3]), red, 5)
#                 if i == 1 or i == 2:
#                     bounding_image = cv2.putText(bounding_image, 'Crosswalk', (c_box[0]+30, c_box[1]-40), font, 2, red, 3)
#                 else:
#                     bounding_image = cv2.putText(bounding_image, 'Crosswalk', (c_box[0]+30, c_box[1]+80), font, 2, red, 3)        
        
#         if pred_idx[image_idx]:
            
#             pred_box = [int(_) for _ in pred[pred_c, 2:6]]
#             pred_box[2] = pred_box[0]+pred_box[2]
#             pred_box[3] = pred_box[1]+pred_box[3]
#             score = pred[pred_c, 6]*100
#             bounding_image = cv2.putText(bounding_image, 'Pred', (pred_box[0], pred_box[3]+60), font, 2, blue, 3)
#             bounding_image = cv2.putText(bounding_image, 'Score: %2d' % (score), (pred_box[0], pred_box[3]+120), font, 2, blue, 3)
            
#             if pred_c >= 1:
#                 x_lu = pred_box[0]
#                 y_lu = pred_box[1]
#                 x_rd = pred_box[2]
#                 y_rd = pred_box[3]
                
#                 x_lu_pr = prev_box[0]
#                 y_lu_pr = prev_box[1]
#                 x_rd_pr = prev_box[2]
#                 y_rd_pr = prev_box[3]                
                
#                 prev_box = pred_box
                
#                 xc_ru = c_box[2]
#                 yc_ru = c_box[1]
                
#                 iou = IoU(c_box, pred_box)
                
# #                 if iou != 0:
#                 if xc_ru >= x_rd and yc_ru <=y_rd or iou !=0:
#                     print(image_idx)
#                     bounding_image = cv2.rectangle(bounding_image, (pred_box[0], pred_box[1]), (pred_box[2], pred_box[3]), blue, 5)
#                     bounding_image = cv2.putText(bounding_image, 'Crossing', (35, 200), font, 3, blue, 4)
#                     cv2.imwrite(os.path.join(OUT_PATH_TRACK, sub_path, image_filename), bounding_image)
#                     pred_c+=1
#                     continue
                
#                 if x_lu - x_lu_pr <= 0:
#                     v = 'LEFT'
# #                     d_cp = math.sqrt((xc_ru - x_lu)**2)
#                     d_cp = math.sqrt((xc_ru - x_rd)**2+(yc_ru-y_rd)**2)+1e-10
#                 elif x_lu - x_lu_pr >0:
#                     v = 'RIGHT'
# #                 elif y_lu - y_lu_pr > 0 and y_rd - y_rd_pr < 0:
# #                     v = v+' & FORWARD'
# #                 elif y_lu - y_lu_pr < 0 and y_rd - y_rd_pr > 0:
# #                     v = v+' & BACKWARD'
# #                 else:
# #                     v = v+' & STOP'
                
#                 d = math.sqrt((x_rd-x_rd_pr)**2 + (y_rd-y_rd_pr)**2)+1e-10
                
#                 if pred_c >=2:
#                     s = abs((d-d_pr)/(image_idx+1-image_idx))
#                     a = (s - s_pr)/(image_idx+1-image_idx)
                    
#                 else:
#                     s = d
#                     a = s
                
#                 t_cp = d_cp/d*fps
#                 d_pr = d
#                 s_pr = s
                
#             else:
#                 prev_box = pred_box
                
                
#                 x_lu = pred_box[0]
#                 y_lu = pred_box[1]
#                 x_rd = pred_box[2]
#                 y_rd = pred_box[3]
                
                
#                 v = 'STOP'
#                 s = 0.
#                 d = 0.
#                 a = 0.
#                 t_cp = 0.
                
            
#             bounding_image = cv2.rectangle(bounding_image, (pred_box[0], pred_box[1]), (pred_box[2], pred_box[3]), blue, 5)
            
#             bounding_image = cv2.putText(bounding_image, 'v: %s    s: %.2f    a: %.2f' % (v, s, a), (35, 200), font, 3, black, 4)
#             bounding_image = cv2.putText(bounding_image, 'Tcp: %.2fsecond' % (t_cp), (35, 300), font, 3, black, 4)
# #             bounding_image = cv2.putText(bounding_image, 'v: %s' % (v), (pred_box[0], pred_box[3]+60), font, 2, blue, 3)
# #             bounding_image = cv2.putText(bounding_image, 's: %.2f' % (s), (pred_box[0], pred_box[3]+120), font, 2, blue, 3)
# #             bounding_image = cv2.putText(bounding_image, 'a: %.2f' % (a), (pred_box[0], pred_box[3]+180), font, 2, blue, 3)
# #             bounding_image = cv2.putText(bounding_image, 'Tcp: %.2fsecond' % (t_cp), (pred_box[0], pred_box[3]+240), font, 2, blue, 3)
#             pred_c+=1
        

#         cv2.imwrite(os.path.join(OUT_PATH_TRACK, sub_path, image_filename), bounding_image)
#     break



NameError: name 'data_list' is not defined

In [5]:
# cross_walk_dict = {
#     'crosswalk': [[35, 582, 928, 836]],
#     'fourway': [[1133, 625, 1920, 1045], [907, 561, 1545, 629], [110, 565, 756, 601], [63, 591, 854, 967]],
#     'night': None
# }

In [8]:
# import re
# path = os.path.join(OUT_PATH_TRACK, 'crosswalk')
# paths = [os.path.join(path , i ) for i in os.listdir(path) if re.search(".png$", i )]
# # pathIn= os.path.join(OUT_PATH_TRACK, 'crosswalk')
# fps = 15
# pathOut = os.path.join(OUT_PATH_TRACK, 'crosswalk_video', 'crosswalk_video_%02d_subset.mp4' % (fps))

# import cv2
# frame_array = []
# for idx , path in enumerate(paths): 
#     if idx >= 90:
#         break
#     img = cv2.imread(path)
#     height, width, layers = img.shape
#     size = (width,height)
#     frame_array.append(img)
# out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
# for i in range(len(frame_array)):
#     # writing to a image array
#     out.write(frame_array[i])
# out.release()